# Run the "Real-time Credit Scoring" tutorial

We'll use the following tutorial as a demonstration.

https://github.com/feast-dev/feast-credit-score-local-tutorial/tree/f43b44b245ae2632b582f14176392cfe31f98da9

## Upload the tutorial source code to the running Feast pod.

Upload the tutorial source code to the running feast pod and extract its contents.

In [ ]:
![ -f f43b44b.tar.gz ] || wget https://github.com/feast-dev/feast-credit-score-local-tutorial/archive/f43b44b.tar.gz
!kubectl cp f43b44b.tar.gz $(kubectl get pods -l 'feast.dev/name=sample-db-ssl' -ojsonpath="{.items[*].metadata.name}"):/feast-data -c online
!kubectl exec deploy/feast-sample-db-ssl -itc online -- rm -rf /feast-data/feast-credit-score-local-tutorial
!kubectl exec deploy/feast-sample-db-ssl -itc online -- mkdir /feast-data/feast-credit-score-local-tutorial
!kubectl exec deploy/feast-sample-db-ssl -itc online -- tar vfx /feast-data/f43b44b.tar.gz -C /feast-data/feast-credit-score-local-tutorial --strip-components 1

## Verify the client `feature_store.yaml`.

Copy the `feature_store.yaml` to the tutorial directory and verify its contents.

In [ ]:
!kubectl exec deploy/feast-sample-db-ssl -itc online -- cp -f /feast-data/postgres_tls_sample/feature_repo/feature_store.yaml /feast-data/feast-credit-score-local-tutorial/feature_repo/feature_store.yaml
!kubectl exec deploy/feast-sample-db-ssl -itc online -- cat /feast-data/feast-credit-score-local-tutorial/feature_repo/feature_store.yaml

## Apply the tutorial feature store definitions

Update the feature store definitions for the tutorial.

In [ ]:
!kubectl exec deploy/feast-sample-db-ssl -itc online -- feast -c /feast-data/feast-credit-score-local-tutorial/feature_repo apply

Load data from feature views into the online store, beginning from either the previous materialize or materialize-incremental end date, or the beginning of time.

In [ ]:
!kubectl exec deploy/feast-sample-db-ssl -itc online -- bash -c 'cd /feast-data/feast-credit-score-local-tutorial/feature_repo && feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")'

## Execute feast commands inside the client Pod

List the registered feast feature views & entities.

In [ ]:
!kubectl exec deploy/feast-sample-db-ssl -itc online -- feast -c /feast-data/feast-credit-score-local-tutorial/feature_repo feature-views list
!kubectl exec deploy/feast-sample-db-ssl -itc online -- feast -c /feast-data/feast-credit-score-local-tutorial/feature_repo entities list

## Train and test the model

Install the required packages.

In [ ]:
!kubectl exec deploy/feast-sample-db-ssl -itc online -- bash -c 'pip install -r /feast-data/feast-credit-score-local-tutorial/requirements.txt'

Train and test the model.

In [ ]:
!kubectl exec deploy/feast-sample-db-ssl -itc online -- bash -c 'cd /feast-data/feast-credit-score-local-tutorial && python run.py'

## Interactive demo (using Streamlit)

In a new terminal, run the following command and leave it active.

```bash
$ kubectl port-forward deploy/feast-sample-db-ssl 8501:8501
```

Start the Streamlit application

In [ ]:
!kubectl exec deploy/feast-sample-db-ssl -itc online -- bash -c 'cd /feast-data/feast-credit-score-local-tutorial && streamlit run --server.port 8501 streamlit_app.py'

Then navigate to the local URL on which Streamlit is being served.

http://localhost:8501